If cellbox package couldn't be installed, the integration can be done manually. To do so:
- in the main.py, change the dir_main to cellbox/cellbox instead of cellbox.
- modify the __init__.py file in cellbox/cellbox to the following format:
    -- from cellbox import config

In [1]:
import sys
import os
import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None
import importlib
MAIN_DIR = 'C:/Users/nourisa/Downloads/testProjs/'
CELLBOX_DIR = os.path.join(MAIN_DIR,'external/CellBox-master')
geneRNI_DIR = os.path.join(MAIN_DIR,'geneRNI')
sys.path.insert(0,geneRNI_DIR)
from geneRNI import tools

In [2]:
# run cellbox with the given config file: it needs python 3.6
%cd C:/Users/nourisa/Downloads/testProjs/external/CellBox-master
%run scripts/main.py -config=configs/Example.random_partition.json

C:\Users\nourisa\Downloads\testProjs\external\CellBox-master


KeyboardInterrupt: 

In [3]:
# process the outputs of cellbox
OUTPUT_DIR = os.path.join(CELLBOX_DIR,'results','Example_RP_bac12a831ae7776664c46d28dd859b74/')
FILE_WS = os.path.join(OUTPUT_DIR,'seed_000','6_best.W.loss.0.034608094468712804.csv') 

# links
from geneRNI import tools

importlib.reload(tools)
ws = pd.read_csv(FILE_WS)
ws_matrix = ws.iloc[:,1:].values
gene_names = ws['0'].values
ws_formatted = tools.links.format(links=ws_matrix,gene_names=gene_names)
ws_formatted

,Regulator,Target,Weight,Sign
0,4EBP1pS65,RbpS807,-0.000087,
1,4EBP1pS65,MAPKpT202,0.010682,
2,4EBP1pS65,MEKpS217,0.028354,
3,4EBP1pS65,S6,-0.007098,
4,4EBP1pS65,PAI-1,0.051288,
...,...,...,...,...
9697,aSRC,aPKC,-0.000000,
9698,aSRC,aSTAT3,-0.000000,
9699,aSRC,amTOR,-0.000000,
9700,aSRC,aPI3K,0.000000,


In [4]:
# read data of cellbox
DATA_DIR = os.path.join(CELLBOX_DIR, 'data')
EXPR_FILE = os.path.join(DATA_DIR,'expr.csv')
GENES_FILE = os.path.join(DATA_DIR,'node_index.csv')
PERT_FILE = os.path.join(DATA_DIR,'pert.csv')

expr_data = np.genfromtxt(EXPR_FILE,delimiter=',')
print(f'n samples: {expr_data.shape[0]}, n genes: {expr_data.shape[1]}')
with open(GENES_FILE) as f:
    gene_names = f.read().split() 
print(f'n genes: {len(gene_names)}')
perts = np.genfromtxt(PERT_FILE,delimiter=',')
print(f'n samples: {perts.shape[0]}, n genes: {perts.shape[1]}')

n samples: 89, n genes: 99
n genes: 99
n samples: 89, n genes: 99


In [5]:
# run geneRNI on cellbox data
from geneRNI import tools
from geneRNI import geneRNI
importlib.reload(tools)
importlib.reload(geneRNI)
Xs, ys = tools.Data.process_static(SS_data=expr_data, gene_names=gene_names, perturbations=perts)
param = dict(estimator_t='RF')
ests, train_scores, links_df, oob_scores, test_scores = geneRNI.network_inference(Xs, ys, gene_names, param)


network inference: train score, mean: 0.9942573033301199 std: 0.004921778303790991
network inference: oob score (only RF), mean: 0.9583104549817966 std: 0.035079425673539305
